**Zachary Qian**

I affirm that I personally wrote the text, code, and comments in this homework assignment.

**Note: I compared outputs and algorithmic concepts with Renzo Tanaka-Wong.**

# HW3

### PageRank

What is the most important website on the internet? Who is the "key player" on a sports team? Which countries are the most central players in the world economy? There is no one correct answer to any of these questions, but there is a most profitable one. [PageRank](https://en.wikipedia.org/wiki/PageRank) is an algorithm for ranking individual elements of complex systems, invited by Sergey Brin and Larry Page. It was the first and most famous algorithm used by the Google Search engine, and it is fair to say that the internet as we know it today would not exist without PageRank. 

In this assignment, we will implement PageRank. There are many good ways to implement this algorithm, but in this assignment we will use our newfound skills with object-oriented programming and iterators. 

### How it works

For the purposes of this example, let's assume that we are talking about webpages. PageRank works by allowing a "random surfer" to move around webpages by following links. Each time the surfer lands on a page, it then looks for all the links on that page. It then picks one at random and follows it, thereby arriving at the next page, where the process repeats. Eventually, the surfer will visit all the pages one or more times. Pages that the surfer visits more frequently have higher *PageRank scores*. Because the surfer moves between linked pages, PageRank expresses an intuitive idea: **important pages are linked to other important pages.** [This diagram](https://en.wikipedia.org/wiki/PageRank#/media/File:PageRanks-Example.jpg) from Wikipedia gives a nice illustration. Note that more important webpages (higher PageRank) tend to be connected to other important webpages. 

<figure class="image" style="width:50%">
  <img src="https://upload.wikimedia.org/wikipedia/en/thumb/8/8b/PageRanks-Example.jpg/1920px-PageRanks-Example.jpg
" alt="A set of 11 circles, with arrows between them. Some of the circles are larger than others, reflecting their high PageRank score. Large circles tend to be linked to other large circles by arrows." width="300px">
  <figcaption><i>A schematic for PageRank. </i></figcaption>
</figure>

### Data

You can complete this assignment using data from one of two sources. 

#### Option 1: Hamilton

This data set comes from the hit Broadway musical "Hamilton."

<figure class="image" style="width:50%">
  <img src="https://pbs.twimg.com/profile_images/1277989982171430914/Oiyyv_7__400x400.jpg" alt="The logo of the musical Hamilton, showing a silhouette dressed in period custom standing on top of a five-pointed star." width="300px">
  <figcaption><i>The Hamilton data set</i></figcaption>
</figure>

The good folks at [The Hamilton Project](http://hamilton.newtfire.org/) analyzed the script for us, obtaining data on **who talks about whom** in each of the show's songs. When character A mentions character B, we'll think of this as a *link* from A to B, suggesting that B might be important. 

If you use this data set, listening to the soundtrack while working is strongly recommended. 

#### Option 2: Global Airline Network


<figure class="image" style="width:50%">
  <img src="https://openflights.org/demo/openflights-routedb.png" alt="A map of the world, with many curved green lines connecting different points on the map. The points are airports, and the curved green lines are flight routes." width="300px">
  <figcaption><i>The global airline network</i></figcaption>
</figure>

Back in the Before Times, lots of people flew on airplanes. This data set includes a "link" from Airport A to Airport B whenever there is a flight from B to A. This data set was collected by the [OpenFlights Project](https://openflights.org/data.html). 

## (A). Define Functions

In this part, all you have to do is hit `shift + enter` on the code block supplied below. This block defines two functions. The first one retrieves the data from the internet and saves it to your local computer, while the second reads in the data, producing a list of tuples. It's not important for you to be familiar with the code in these functions right now -- we'll discuss them early in Week 4. 

In [1]:
import urllib
import csv

def retrieve_data(url):
    """
    Retrieve a file from the specified url and save it in a local file 
    called data.csv. The intended values of url are: 
    
    1. https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv
    2. https://philchodrow.github.io/PIC16A/homework/HW3-flights-data.csv
    """
    
    # grab the data and parse it
    filedata = urllib.request.urlopen(url) 
    to_write = filedata.read()
    
    # write to file
    with open("data.csv", "wb") as f:
        f.write(to_write)
        
def read_data(path):
    """
    read downloaded data from a .csv file, and return a list of tuples. 
    each tuple represents a link between states. 
    """
    with open(path, "r") as f:
        reader = csv.reader(f)
        return [(row[0], row[1]) for row in list(reader)]

## (B). Grab the Data

The data live at the following URLs: 

- **Hamilton**:  `https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv`
- **Airline**:   `https://philchodrow.github.io/PIC16A/homework/HW3-flights-data.csv` 

In each data set, each row corresponds to a "link" between objects. In Hamilton, the pairs have format `mentioner, mentioned` while in the airline network the rows have format `origin, destination.`

Pick one of these data sets, and set the variable `url` appropriately by uncommenting one of the two lines below. Then, call `retrieve_data()` and `read_data()`. Create a variable called `data` to hold the return value of `read_data()`. 

#### Your solution

In [2]:
# uncomment the second line if you'd prefer to 
# work with the flights data. 
#url = "https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv"
url = "https://philchodrow.github.io/PIC16A/homework/HW3-flights-data.csv"
# url = "https://philchodrow.github.io/PIC16A/homework/HW3-flights-data.csv"

# Call your functions below
retrieve_data(url)
data = read_data("data.csv")  #simple call to set read_data return obj to "data"

In [4]:
str(data) #structure of data, for my own use

"[('KZN', 'AER'), ('LPA', 'GVA'), ('ZRH', 'LPA'), ('GVA', 'TFS'), ('LPA', 'ZRH'), ('SIN', 'BKK'), ('SIN', 'HAK'), ('SIN', 'HGH'), ('SIN', 'HKG'), ('MNL', 'KIX'), ('TPE', 'KIX'), ('SIN', 'KUL'), ('KIX', 'MNL'), ('SIN', 'MNL'), ('SIN', 'SGN'), ('BKK', 'SIN'), ('HAK', 'SIN'), ('HGH', 'SIN'), ('HKG', 'SIN'), ('KUL', 'SIN'), ('MNL', 'SIN'), ('SGN', 'SIN'), ('TPE', 'SIN'), ('KIX', 'TPE'), ('SIN', 'TPE'), ('MCO', 'FLL'), ('TPA', 'FLL'), ('FLL', 'MCO'), ('FLL', 'TPA'), ('CMN', 'BCN'), ('CMN', 'BRU'), ('BCN', 'CMN'), ('BRU', 'CMN'), ('LGW', 'CMN'), ('LYS', 'CMN'), ('SAW', 'CMN'), ('CMN', 'LGW'), ('CMN', 'LYS'), ('CMN', 'SAW'), ('CKG', 'CAN'), ('CTU', 'CAN'), ('CKG', 'CGO'), ('CTU', 'CGO'), ('HGH', 'CGO'), ('HRB', 'CGO'), ('KMG', 'CGO'), ('URC', 'CGO'), ('CAN', 'CKG'), ('CGO', 'CKG'), ('CSX', 'CKG'), ('CTU', 'CKG'), ('DLC', 'CKG'), ('HAK', 'CKG'), ('HGH', 'CKG'), ('KMG', 'CKG'), ('NKG', 'CKG'), ('NNG', 'CKG'), ('PEK', 'CKG'), ('PVG', 'CKG'), ('SYD', 'CKG'), ('SYX', 'CKG'), ('SZX', 'CKG'), ('TNA'

## (C). Examine the structure of the data

This would also be a good time to inspect the data to make sure you understand how it is structured. Write a function `describe(n)` that describes the meaning of the `n`th row of the data set you chose. In the Hamilton data set, your function should do the following: 

```python
describe(5)

# output
"Element 5 of the Hamilton data set is ('burr', 'betsy'). This means that Burr mentions Betsy in a song." 
```

In context of the airline flights data, your function should instead do this: 

```python
describe(5)

# output
"Element 5 of the flights data set is ('SIN', 'BKK'). This means that there is a flight from BKK to SIN." 
```

Please attend to capitalization and formatting. While the standard string concatenation operator `+` is completely fine for this task, the fancy `str.format()` function may make your code somewhat simpler. [This page](https://realpython.com/python-formatted-output/) has some useful examples in case you'd like to try this. 


#### Your Solution

In [3]:
def describe(n):
    """
    Describes data explaining the actual element of the data set then explaining where the flight was from and where it flew to.
    """
    txt = "Element {} of the flight data set is ('{}','{}'). This means that there is a flight from {} to {}.".format(
        n,data[n][0],data[n][1],data[n][1],data[n][0]) #using format and literally doing the exact same format as asked
    return txt
    

In [8]:
describe(5)

"Element 5 of the flight data set is ('SIN','BKK'). This means that there is a flight from BKK to SIN."

## (D). Data to Dictionary

Write a function called `data_to_dictionary` that converts the data into a dictionary such that: 

1. There is a single key for each character (in Hamilton) or airport (in flights). 
2. The value corresponding to each key is a list of the characters/airports to which that key links. The list should contain repeats if there are multiple links. 

Here's an example of the desired behavior on a fake data set. 

```python
data = [("a", "b"), 
        ("a", "b"), 
        ("a", "c"),
        ("b", "c"),
        ("b", "a")]
        
data_to_dictionary(data)

# output
{"a" : ["b", "b", "c"], "b" : ["a", "c"]}

```

#### Your Solution

In [4]:
def data_to_dictionary(data):
    """
    Used to convert the list of tuples into a dictionary. 
    """
    d = {} #initialize dictionary
    for route in data: #check every plane route tuple in data
        if route[0] not in d.keys(): #if the destination is not already in the dict
            d[route[0]] =  [route[1]] #add it in list() form
        else:
            d[route[0]].append(route[1]) #otherwise it does exist; therefore, just append to the existing key
    return d

In [10]:
test = data_to_dictionary(data)
#test

In [11]:
test #for my own use

{'KZN': ['AER'],
 'LPA': ['GVA',
  'ZRH',
  'CGN',
  'CGN',
  'DUS',
  'HAM',
  'MUC',
  'TXL',
  'VIE',
  'ZRH',
  'CMN',
  'MAD',
  'DUS',
  'FRA',
  'HAM',
  'MUC',
  'STR',
  'ARN',
  'CGN',
  'CPH',
  'HAM',
  'LGW',
  'MUC',
  'OSL',
  'DUB',
  'BCN',
  'BHX',
  'DUB',
  'EDI',
  'LTN',
  'MAD',
  'MAN',
  'STN',
  'VIE',
  'DUS',
  'BRU',
  'AMS',
  'TFS',
  'AGP',
  'ALC',
  'BCN',
  'MAD',
  'ORY',
  'MAN',
  'ZRH',
  'ACE',
  'CMN',
  'LIS',
  'TFS',
  'MAD',
  'VIE',
  'ACE',
  'PRG',
  'LIS',
  'CDG',
  'OSL',
  'LGW',
  'MAN',
  'BHX',
  'LGW',
  'LTN',
  'MAN',
  'STN',
  'LGW',
  'MAN',
  'LGW',
  'MAD',
  'AGP',
  'BCN',
  'ORY',
  'CGN',
  'DUS',
  'FRA',
  'MUC',
  'STR',
  'BHX',
  'MAN'],
 'ZRH': ['LPA',
  'AYT',
  'CGN',
  'HAM',
  'JFK',
  'LHR',
  'PHL',
  'ACE',
  'ALC',
  'CTA',
  'DUS',
  'HEL',
  'HER',
  'LHR',
  'LPA',
  'MAD',
  'PMI',
  'TFS',
  'TXL',
  'VIE',
  'YUL',
  'YYZ',
  'CDG',
  'BOM',
  'DEL',
  'CMN',
  'HEL',
  'LIS',
  'FCO',
  'LHR',
  'RI

## (E). Define a PR_DiGraph class

A **directed graph**, or DiGraph, is just a set of arrows ("edges") between objects ("nodes"). It is a natural way to represent data that represents one-way relationships, such as links from one webpage to another or mentions of one character by another. We already saw a directed graph above when we introduced the idea of PageRank. Here's a paired-down example. 

<figure class="image" style="width:50%">
  <img src="https://computersciencewiki.org/images/c/c6/Directed_graph.png" alt="Six circles, representing nodes, labeled A through F. There are directed arrows between certain pairs of nodes." width="300px">
  <figcaption><i>Example of a directed graph. </i></figcaption>
</figure>

Implement a `PR_DiGraph` class with a custom `__init__()` method and a `linked_by()` method. The `__init__()` method should accept two arguments: `data` and `iteration_limit`. The `__init__()` method should then construct an instance variable `self.link_dict` which is simply the output of `data_to_dictionary` applied to the argument `data`. `__init__()` should also construct an instance variable `self.iteration_limit`, which simply takes the value of `iteration_limit` supplied to `__init__()`. Don't worry about that one for now. 

Then, define a method `self.linked_by(x)` which, when called, returns the value `self.link_dict[x]`.  

Finally, add an `__iter__` method, which returns an object of class `PR_Iterator`. We will define this class in the next part.  

Example session (using Hamilton): 

```python 
D = PR_DiGraph(data, iteration_limit = 10000)
D.linked_by('peggy')

# output
['peggy', 'schuylerSis']
```

#### Your Solution

In [81]:
class PR_DiGraph:
    
    """
    Construct a directed graph, which has a constructor (__init__()), linked_by(), and __iter__() fxns.
    Recall from Math 61 & 182 that a directed graph is one that typically has arrows to show direction through edges as it goes node-to-node
    """
    
    # implement with custom __init__() method and linked_by() method
    
    def __init__(self, data, iteration_limit): #initialize and make a constr
        self.data = data #set our data to data param
        self.iteration_limit = iteration_limit  #ibid constructor stuff, just like the previous line
        self.link_dict = data_to_dictionary(data) #ibid constructor stuff
    
    
    def linked_by(self,x): #define linked_by, which will return the the linked nodes/airports the current airport is linked to
        return self.link_dict[x]
    
    def __iter__(self): #iterator, returns PR_iterator object
        return PR_Iterator(self)
        
        

## (F). Implement PR_Iterator

Define a `PR_Iterator` class with a custom `__next__()` method. 

The `__init__` method of this class should create instance variables to store the `PR_DiGraph` object from which it was constructed; a counter `i`, starting at 0, to log the number of steps taken, and a `current_state` variable whose value is one of the keys of the `link_dict` of the `Pr_DiGraph`. You can choose its initial value arbitrarily; in my solution code I chose `self.current_state = "hamilton"`.

We are going to use iteration to implement the PageRank algorithm. This  means we are going to imagine a surfer who is following the links in our data set. **Implement the following two methods:** 

1. `follow_link()`. 
    1. Pick a random new character mentioned by the current character, or new airport served by the current airport. Let's call this `next_state`. 
    2. If `next_state != current_state`, set `current_state` to `next_state`. 
    3. Otherwise (if `next_state == current_state`), teleport (see below). 
    4. You might run into `KeyError`s, in which case you should again teleport (use a `try-except` block). 
2. `teleport()`. 
    1. Set the current state to a new state (key of the link dict) completely at random. 

*Hint*: use `random.choice` from the `random` module to choose elements of lists. 

Finally, **implement** `__next__()`. `__next__()` should do `follow_link()` with 85% probability, and do `teleport()` with 15% probability. You should also define a custom `StopIteration` condition to ensure that only as many steps are taken as the `iteration_limit` supplied to the `PR_DiGraph` initializer.

1. To do something with 85% probability, use the following: 

```
if random.random() < 0.85:
    # do the thing
else:
    # do the other thing
```


#### Example Usage

After you define your class, run the following code and show that it works. Note: your precise sequence may be different from mine. 

```python
D = PR_DiGraph(data, iteration_limit = 5)
for char in D:
    print(char)
    
following link : current state = burr
following link : current state = washington
following link : current state = burr
following link : current state = hamilton
teleporting    : current state = washington
```

I have added printed messages here for you to more clearly see what should be happening, but it is not necessary for you to do this. It is sufficient for your output to look like: 

```python
D = PR_DiGraph(data, iteration_limit = 5)
for char in D:
    print(char)
    
burr
washington
burr
hamilton
washington
```

#### Your Solution

Finally, **implement** `__next__()`. `__next__()` should do `follow_link()` with 85% probability, and do `teleport()` with 15% probability. You should also define a custom `StopIteration` condition to ensure that only as many steps are taken as the `iteration_limit` supplied to the `PR_DiGraph` initializer.

1. To do something with 85% probability, use the following: 

```
if random.random() < 0.85:
    # do the thing
else:
    # do the other thing
```


In [91]:
import random

class PR_Iterator:
    """
    Build Page Rank Iterator Through the actual Directed Graph
    """
    
    def __init__(self, digraph): # data, iteration_limit
        self.digraph = digraph #set our digraph to the one in the parameter
        self.link_dict = data_to_dictionary(digraph.data) #convert our digraph.data to linked dict using our old method data_to_dictionary()
        self.current_state = "peopleNeedToStopProjectingZukoOntoThemselves" #initialization is arbitrary, message stays
        self.i = 0 #create iterator index at 0
    
    def follow_link(self):
        """
        picks new airport served by current airport defined as next_state
        
        use try-except blocks
        
        """
        try:
            self.next_state = random.choice(self.link_dict[self.current_state]) #set our next state to some randomly linked state
            if self.next_state != self.current_state: #if they aren't the same state, then set current_state to the new one
                self.current_state = self.next_state
            else:
                self.teleport() #otherwise, teleport
                    
        except KeyError:  #raise a KeyError, and then proceed to teleport
            self.teleport()
            
        return self.current_state #return current state
        
        

    def teleport(self):
        """
        Set the current state to a new state (key of the link dict) completely at random.
        """
        
        self.current_state = random.choice(list(self.digraph.link_dict.keys())) #set rand as a random choice of keys
        return self.current_state #return our curren state
        
    def __next__(self):
        """
        iterate through, but first checks to see if the index is greater than the iteration limit, which then raises a StopIteration error
        
        then, we check to see randomly if some n < 0.85 ==> return follow_link() so that it goes through the next state at random
        
        otherwise, we will teleport
        
        """
        if self.i > self.digraph.iteration_limit:  #first, check if i reached iteration limit
            raise StopIteration #stop if you do
        self.i+=1 #increment i by 1
        if random.random() < 0.85:
            return self.follow_link()
        else:
            return self.teleport()

In [92]:
# run the below
D = PR_DiGraph(data, iteration_limit = 5)
for char in D:
    print(char)

MAD
MIA
ZRH
DOH
MCO
CUN


In [90]:
D.linked_by("JFK")

['MEX',
 'MSY',
 'AMM',
 'AUH',
 'BCN',
 'BOS',
 'BWI',
 'CDG',
 'CLT',
 'CUN',
 'DCA',
 'DFW',
 'DOH',
 'DUB',
 'DUS',
 'FCO',
 'GRU',
 'HEL',
 'HKG',
 'LAS',
 'LAX',
 'LHR',
 'LIM',
 'MAD',
 'MAN',
 'MCO',
 'MIA',
 'MXP',
 'NRT',
 'ORD',
 'ORY',
 'PHX',
 'SAN',
 'SEA',
 'SFO',
 'SJU',
 'TPA',
 'TXL',
 'YUL',
 'YYC',
 'YYZ',
 'ZRH',
 'DUS',
 'TXL',
 'YYZ',
 'ATL',
 'BCN',
 'BRU',
 'CDG',
 'DUB',
 'FRA',
 'LHR',
 'MAD',
 'MSY',
 'NCE',
 'DEL',
 'LHR',
 'ATL',
 'CUN',
 'MEX',
 'LAX',
 'SAN',
 'SEA',
 'SFO',
 'CMN',
 'BOG',
 'BCN',
 'CDG',
 'DUB',
 'FCO',
 'HEL',
 'LHR',
 'MAD',
 'MAN',
 'MXP',
 'SJU',
 'YUL',
 'YYZ',
 'ATL',
 'CDG',
 'FCO',
 'MEX',
 'MSY',
 'MXP',
 'VCE',
 'YUL',
 'BOS',
 'CLT',
 'CUN',
 'DEN',
 'DXB',
 'FLL',
 'IAD',
 'JNB',
 'LAS',
 'LAX',
 'MCO',
 'MSY',
 'MXP',
 'ORD',
 'PDX',
 'PHX',
 'SAN',
 'SEA',
 'SFO',
 'SJU',
 'SLC',
 'TPA',
 'BCN',
 'CDG',
 'DUB',
 'FCO',
 'HEL',
 'LHR',
 'MAD',
 'MAN',
 'MXP',
 'ORY',
 'TPE',
 'PEK',
 'KIX',
 'PTY',
 'HKG',
 'YVR',
 'AMS',


## (G). Compute PageRank

Finally, we are ready to compute the PageRank in our data set. Initialize a `PR_DiGraph` with a large iteration limit (say, 1,000,000). Use a `for`-loop to allow your surfer to randomly move through the data set. The number of times that the surfer visits state `x` is the PageRank score of `x`. 

Create a `dict` which logs how many times a given state appears when iterating through the `PR_Digraph`. So, this dictionary holds the PageRank score of each state. 

#### Your Solution

In [93]:
pr_airport = PR_DiGraph(data, iteration_limit = 1000000) #initialize the directed graph

pageRankLog = {} #instantiate dict

for j in pr_airport.data: #first we have to fill in the actual dictionary with entries of the airports
    if j[0] not in pageRankLog.keys(): #if it isn't already in the keys
        pageRankLog[j[0]] = 0 #add it and set all initial values to 0

for i in pr_airport: #then proceed to surf the states and compute PR score
    if i in pageRankLog.keys(): #if we find a match, add to PR score initalized as 0
        pageRankLog[i] +=1

In [75]:
#pageRankLog

## (H).  Display Your Result

Use your favorite approach to show the results in sorted format, descending by PageRank score. The entries at the top should be the entries with highest PageRank. What are the most important elements in the data set? 

You may show either the complete list or just the top 10. 

Check your code by comparing your top 10 to mine. Because we are using a randomized algorithm, your results will not agree exactly with mine, but they should be relatively close. If your top 10 list is very different, then you might want to revisit your previous solutions. 

For Hamilton, my top 10 were: 

```
[('hamilton', 166062),
 ('burr', 99180),
 ('washington', 92246),
 ('jefferson', 72450),
 ('eliza', 51485),
 ('angelica', 48042),
 ('madison', 37421),
 ('lafayette', 34297),
 ('lee', 33678),
 ('jAdams', 31121)]
```

For the flights data, my top 10 were: 

```
[('LHR', 18043), # London Heathrow
 ('ATL', 16370), # Atlanta
 ('JFK', 14795), # New York JFK
 ('FRA', 14156), # Frankfurt
 ('CDG', 14073), # Charles de Gaulle (Paris)
 ('LAX', 13199), # Los Angeles
 ('ORD', 12915), # Chicago O'Hare
 ('PEK', 12525), # Beijing
 ('AMS', 12410), # Amsterdam Schiphol
 ('PVG', 11517)] # Shanghai
```
#### Your solution

In [94]:
sorted(pageRankLog.items(), key=lambda x:x[1], reverse=True) 
#sort based on a lambda function for the 2nd index of the tuple, in descending order

[('LHR', 18261),
 ('ATL', 16287),
 ('JFK', 14901),
 ('CDG', 14350),
 ('FRA', 14292),
 ('ORD', 13231),
 ('LAX', 13112),
 ('PEK', 12573),
 ('AMS', 12411),
 ('PVG', 11340),
 ('MUC', 10942),
 ('FCO', 10783),
 ('HKG', 10740),
 ('SIN', 10586),
 ('ICN', 10529),
 ('BCN', 10395),
 ('MAD', 9792),
 ('BKK', 9695),
 ('NRT', 9613),
 ('DXB', 9417),
 ('DFW', 9247),
 ('YYZ', 9170),
 ('CAN', 9037),
 ('VIE', 8819),
 ('CTU', 8786),
 ('HGH', 8642),
 ('BRU', 8613),
 ('ZRH', 8592),
 ('SFO', 8420),
 ('MAN', 8381),
 ('XIY', 8243),
 ('AUH', 8141),
 ('LGW', 8123),
 ('TPE', 8088),
 ('MIA', 8038),
 ('EWR', 7989),
 ('CPH', 7864),
 ('DUS', 7787),
 ('KMG', 7721),
 ('CKG', 7605),
 ('XMN', 7530),
 ('IST', 7517),
 ('PHL', 7130),
 ('DEN', 7117),
 ('MXP', 7044),
 ('LIS', 6945),
 ('TXL', 6854),
 ('KUL', 6659),
 ('BOS', 6617),
 ('SZX', 6605),
 ('IAD', 6604),
 ('LAS', 6587),
 ('SEA', 6497),
 ('PRG', 6491),
 ('CGO', 6370),
 ('ARN', 6370),
 ('PMI', 6305),
 ('MCO', 6275),
 ('DTW', 6262),
 ('PHX', 6257),
 ('WUH', 6254),
 ('TAO',

## (I). Submit!

Check that your code is appropriately documented (comments and docstrings), and turn it in. 

In [77]:
#ok, i will